<a href="https://colab.research.google.com/github/rgclapp007/gp211-class-notebooks/blob/pef/texture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install "sep_plot @ git+http://zapad.stanford.edu/bob/pySepPlot.git@2bffacb9fb36963339a0834c2b04a0aedff91db4"


In [ ]:
import copy
import numpy as np
from sep_python.sep_vector import FloatVector,get_sep_vector
class boxFilter(FloatVector):
    """Class for defining a filter on a box"""
    def __init__(self,sh,mask,zero_lag,vals=None,space_only=False):
        """
        Create a filter that is a n-d cube with some of the fixed (probably zeroed
        
        sh   - N-d arry of shape
        zero_lag - Location of the zero lag in the filter
        
        mask - Mask for filter coefs n-d array (should be 0 or 1
        vals - Value for filter coeficients
        
        space_only - Whether or not actually have storage
        """
        if tuple(sh)!= mask.shape:
            raise Exception(f"Mask {mask.shape} and filter {sh} shape must be the same")
        
        if vals is not None:
            if vals.shape != sh:
                raise Exception("vals not the same shape")
        sh=np.asarray(sh)[::-1]
        hyper=Hypercube.set_with_ns(sh)
        super().__init__(hyper,vals=vals)    
        self.mask=copy.deepcopy(mask)   
        self.zeroL=np.asarray(zero_lag)
        
    @classmethod
    def PEF(self,sh,one_loc):
        if len(sh) != len(one_loc):
            raise Exception("One location is not the same size as box filter shape")
        b=1
        one_1d=0
        for filt_s, one_s in zip(reversed(sh),reversed(one_loc)):
            if one_s < 0 or one_s >= filt_s:
                raise Exception("Illegal locaiton for one in the filter")
            one_1d+=one_s*b
            b*=filt_s
        
        msk=np.ones(sh,dtype=np.float32)
        m=np.ravel(msk)
        vals=np.zeros(sh,dtype=np.float32)
        v=np.ravel(vals)
        v[:one_1d]=0
        m[:one_1d+1]=0
        v[one_1d]=1
        return boxFilter(sh,msk,one_loc,vals=vals)

                     
    def clone(self):
        """Return a clone of the helix filter"""
        x=boxFilter(self.get_nd_array().shape,self.mask,self.zeroL,vals=self.get_nd_array())
        return x
    
    def cloneSpace(self):
        x=boxFilter(self.get_nd_array().shape,self.mask,self.zeroL,space_only=True)
        return x

    

In [ ]:
import numba
from genericSolver.pyOperator import Operator
class convOpAdjData(Operator):
    def __init__(self,model,data,filt):
        if not model.checkSame(data) and not isinstance(model,FloatVector):
            raise Exception("Expecting model and data to be the same shape and float vectors")
        self.filt=filt.clone()
        
        if len(filt.get_nd_array().shape) >len(model.get_nd_array().shape):
            raise Exception("Filter must be same number of dimensions or smaller than model/data")
        super().__init__(model,data)
        
        
    
    def forward(self,add,model,data):
        self.checkDomainRange(model,data)
        if not add:
            data.zero()
        m=model.get_nd_array()
        d=data.get_nd_array()
        f=self.filt.get_nd_array()
        if len(m.shape)==2:
            if len(f.shape)==2:
                forward_2_2(m,f,d,self.filt.zeroL)
            else:
                forward_2_1(m,f,d,self.filt.zeroL)
        else:
            forward_1_1(m,f,d,self.filt.zeroL)
        
        
    
    def adjoint(self,add,model,data):
        self.checkDomainRange(model,data)
        if not add:
            model.zero()
        m=model.get_nd_array()
        d=data.get_nd_array() 
        f=self.filt.get_nd_array()
        

        if len(m.shape)==2:
            if len(f.shape)==2:
                adjoint_m_2_2(m,f,d,self.filt.zeroL)
            else:
                adjoint_m_2_1(m,f,d,self.filt.zeroL)
        else:
            adjoint_m_1_1(m,f,d,self.filt.zeroL)
        
class convOpAdjFilter(Operator):
    def __init__(self,filt,data,model):
        if not model.checkSame(data) and not isinstance(model,FloatVector):
            raise Exception("Expecting model and data to be the same shape and float vectors")
        self.model=copy.deepcopy(model)
        
        if len(filt.get_nd_array().shape) >len(model.get_nd_array().shape):
            raise Exception("Filter must be same number of dimensions or smaller than model/data")
        super().__init__(filt,data)
        
        
    
    def forward(self,add,filt,data):

        self.checkDomainRange(filt,data)
        if not add:
            data.zero()
        m=self.model.get_nd_array()
        d=data.get_nd_array()
        ftemp=filt.clone()
        f=ftemp.get_nd_array()
        f[:]=f[:]*filt.mask[:]

        if len(m.shape)==2:
            if len(f.shape)==2: 
               forward_2_2(m,f,d,filt.zeroL)
            else:
                forward_2_1(m,f,d,filt.zeroL)
        else:
            forward_1_1(m,f,d,filt.zeroL)
        
        
    def adjoint(self,add,filt,data):
        self.checkDomainRange(filt,data)
        if not add:
            filt.zero()
        m=self.model.get_nd_array()
        d=data.get_nd_array() 
        ftemp=filt.clone()
        f=ftemp.get_nd_array()
        ftemp.zero()
        if len(m.shape)==2:
            if len(f.shape)==2:
                adjoint_f_2_2(m,f,d,filt.zeroL)
            else:
                adjoint_f_2_1(m,f,d,filt.zeroL)
        else:
            adjoint_f_1_1(m,f,d,filt.zeroL)
        f[:]=f[:]*filt.mask[:]
        filt.scale_add(ftemp)
                
@numba.njit()
def forward_2_2(m ,f,d,zero): 
    for i2 in range(f.shape[0]-zero[0]-1,m.shape[0]-zero[0]):
        for i1 in range(f.shape[1]-zero[1]-1,m.shape[1]-zero[1]):
            for if2 in range(0,f.shape[0]):
                for if1 in range(0,f.shape[1]):
                    d[i2,i1]+=f[if2,if1]*m[i2-if2+zero[0] ,i1-if1+zero[1]]

    
@numba.njit()
def forward_2_1(m,f,d,zero):               
    for i2 in range(m.shape):
        for i1 in range(f.shape[0]-zero[0]-1,m.shape[1]-zero[0]):
                for if1 in range(0,f.shape[1]):
                    d[i2,i1]+=f[if1]*m[i2 ,i1-if1+zero[0]]
                                 
@numba.njit()
def forward_1_1(m,f,d,zero):             
    for i1 in range(f.shape[0]-zero[0]-1,m.shape[0]-zero[0]):
            for if1 in range(0,f.shape[1]):
                d[i1]+=f[if1]*m[i1-if1+zero[0]]

@numba.njit()
def adjoint_m_2_2(m,f,d,zero):          
    for i2 in range(f.shape[0]-zero[0]-1,m.shape[0]-zero[0]):
        for i1 in range(f.shape[1]-zero[1]-1,m.shape[1]-zero[1]):
            for if2 in range(0,f.shape[0]):
                for if1 in range(0,f.shape[1]):
                    m[i2-if2+zero[0] ,i1-if1+zero[1]]+=d[i2,i1]*f[if2,if1]

@numba.njit()
def adjoint_m_2_1(m,f,d,zero):  
    for i2 in range(m.shape[0]):
        for i1 in range(f.shape[0]-zero[0]-1,m.shape[1]-zero[0]):
                for if1 in range(0,f.shape[0]):
                    m[i2,i1-if1+zero[1]]+=d[i2,i1]*f[if1]
  
@numba.njit()
def adjoint_m_1_1(m,f,d,zero):
    for i1 in range(f.shape[0]-zero[0]-1,m.shape[0]-zero[0]):
            for if1 in range(0,f.shape[0]):
                m[i1-if1+zero[1]]+=d[i1]*f[if1]
                
@numba.njit()
def adjoint_f_2_2(m,f,d,zero):    

    for i2 in range(f.shape[0]-zero[0]-1,m.shape[0]-zero[0]):
        for i1 in range(f.shape[1]-zero[1]-1,m.shape[1]-zero[1]):
            for if2 in range(0,f.shape[0]):
                for if1 in range(0,f.shape[1]):
                    f[if2,if1]+=m[i2-if2+zero[0] ,i1-if1+zero[1]]*d[i2,i1]
@numba.njit()
def adjoint_f_2_1(m,f,d,zero):  
    for i2 in range(m.shape[0]):
        for i1 in range(f.shape[0]-zero[0]-1,m.shape[1]-zero[0]):
                for if1 in range(0,f.shape[0]):
                    f[if1]+=m[i2,i1-if1+zero[1]]*d[i2,i1]
  
@numba.njit()
def adjoint_f_1_1(m,f,d,zero):
    for i1 in range(f.shape[0]-zero[0]-1,m.shape[0]-zero[0]):
            for if1 in range(0,f.shape[0]):              
                f[if1]+=m[i1-if1+zero[1]]*d[i1]

In [ ]:
textures=["brick","granite","ridges","sfbay","twofreq","brickpef","herr","ridgespef","skull","wood","fabric","polygons","sepele","stanford.tree","woodpef"]

In [ ]:
import sep_python.modes
import numpy as np
from sep_python.hypercube import Hypercube
from sep_python.sep_vector import FloatVector
io=sep_python.modes.default_io
#vec=io.vector_from_storage("./galilee.H")

In [ ]:
from genericSolver.pyProblem import ProblemL2Linear,ProblemL2LinearReg
from genericSolver.pyLinearSolver import LCGsolver
from genericSolver.pyStopper import BasicStopper 
from sep_plot import Grey
def findPef(texture,sh,zero):
    filt=boxFilter.PEF(sh,zero)
    data=texture.clone()
    op=convOpAdjFilter(filt,data,data)
    data.scale(-1.)
    prob=ProblemL2Linear(filt,data,op)
    stop=BasicStopper(niter=1000)
    solve=LCGsolver(stop)
    solve.setDefaults(save_model=True)
    solve.run(prob,verbose=True)
    return prob.model,prob.res


def do_texture(nm):
    vec=io.vector_from_storage(f"./data/textures/{nm}.H")
    filt,rr=findPef(vec,(2,9),(0,4))
    op=convOpAdjData(vec,vec,filt)
    res=vec.clone()
    op.forward(False,vec,res)
    return Grey(vec)+Grey(res)+Grey(rr)

In [ ]:
nm="twofreq"
do_texture(nm)
#vec=io.vector_from_storage(f"./data/textures/{nm}.H")
#Grey(vec)